# Load Tensor Flow Lite Test Model And Run Inference on Test Data
Test Model and Data copied from here
https://jira01.corp.linkedin.com:8443/browse/MID-5889?focusedCommentId=22945746&page=com.atlassian.jira.plugin.system.issuetabpanels%3Acomment-tabpanel#comment-22945746

In [46]:
import numpy as np
import tensorflow as tf
import pandas as pd
import json

# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="native_tflite_model_from_mid_dummy_data_20220621.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Load test data - note: Test data includes a prediction column already.
# Compare our inference with the included inference
df = pd.read_json("test_data_20220621.txt", orient="records", lines=True)
columns_to_drop = ['label']
df.drop(columns=columns_to_drop, inplace=True)
predictions = df.pop("prediction")

# infer score for each test and write scores to file
inferred_results = {}
for i in range(len(df)):
  input_data = np.array(df.loc[[i]], dtype= np.float32)
  interpreter.set_tensor(input_details[0]['index'], input_data)
  interpreter.invoke()
  output_data = interpreter.get_tensor(output_details[0]['index'])
  inferred = output_data[0][0]
  #print ("Test ", i, ", predicted ", np.float32(predictions.loc[i]), ", inferred ", inferred)
  inferred_results[i]=inferred
print (json.dump(inferred_results))
with open('python-inferred-results.json', 'w') as outfile:
    outfile.write(json.dump(inferred_results))


TypeError: dump() missing 1 required positional argument: 'fp'